In [3]:
# 모든 행 출력
pd.set_option('display.max_rows', None)

# 모든 열 출력
pd.set_option('display.max_columns', None)

# 열 너비 자동 조정 (가독성 향상)
pd.set_option('display.width', None)

# 각 컬럼에 표시할 최대 문자 수 늘리기 (딕셔너리 컬럼 등)
pd.set_option('display.max_colwidth', None)


In [1]:
import pandas as pd

# 경로 설정
file_path = "gs://codeit-project/hackle/2023-06-29.parquet"
key_path = "/home/leesh/codeit_project/codeit-project-docker/config/key.json"

# GCS에서 parquet 불러오기
df_230629 = pd.read_parquet(file_path, storage_options={"token": key_path})

# 확인
print(df_230629.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839972 entries, 0 to 3839971
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Asia/Seoul         object 
 1   event_key          object 
 2   session_id         object 
 3   user_id            object 
 4   value              float64
 5   user_properties    object 
 6   hackle_properties  object 
 7   event_properties   object 
 8   id                 object 
 9   device_id          object 
dtypes: float64(1), object(9)
memory usage: 293.0+ MB
None


In [4]:
df_230629.head()

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-06-30T00:00:00.027Z,$session_end,kaNt4iJwceeJKD9JaQorHufsd9s1,kaNt4iJwceeJKD9JaQorHufsd9s1,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion': '13', 'devicevendor': 'samsung', 'versionname': '1.2.15', 'osname': 'Android', 'platform': 'Mobile', 'devicemodel': 'SM-A536N'}",[],8857ee93-3cf1-4f99-99c5-7411f60befc4,6a9bbd4a-7427-401c-80b3-6d82161ccb3b
1,2023-06-30T00:00:00.111Z,view_timeline_tap,FKykiqRYlqbIBV2zfzs5GP1VBd93,FKykiqRYlqbIBV2zfzs5GP1VBd93,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion': '12', 'devicevendor': 'samsung', 'versionname': '1.2.16', 'osname': 'Android', 'platform': 'Mobile', 'devicemodel': 'SM-N971N'}",[],1f029b7d-3717-441c-ae75-54c5cceae1d7,dfea6e11-6022-4bfa-800b-61b87dff0ac1
2,2023-06-30T00:00:00.133Z,click_question_open,5aXHRNZW53ONP3R6IUFZyT0HuVW2,5aXHRNZW53ONP3R6IUFZyT0HuVW2,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone13,1'}",[],737f7f50-bb39-4d70-85e6-1a7b594cd087,ebb00523-9d18-445c-97ee-239dd8a68e59
3,2023-06-30T00:00:00.183Z,view_home_tap,Cmni80eIOEQ8cnKxZR3wNvuUlYu2,Cmni80eIOEQ8cnKxZR3wNvuUlYu2,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion': '13', 'devicevendor': 'samsung', 'versionname': '1.2.16', 'osname': 'Android', 'platform': 'Mobile', 'devicemodel': 'SM-M536S'}",[],18686219-c3e8-4283-95a7-f6b79dfcae1c,9652674b-57ea-4d32-8f9c-21c11308aaef
4,2023-06-30T00:00:00.211Z,view_questions_tap,3evjuZxKXUThPQjb1aVenDTzbXZ2,3evjuZxKXUThPQjb1aVenDTzbXZ2,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osversion': '16.5.1', 'devicevendor': 'Apple', 'versionname': '1.2.15', 'osname': 'iOS', 'platform': 'Mobile', 'devicemodel': 'iPhone14,4'}",[],a3cf0763-1ce9-4925-8ca8-d09b03377a44,40b75183-c3f4-4173-ab6f-9bc64cbc70de


In [5]:
df_simple = df_230629.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','device_id', 'event_key', 'devicemodel', 'language']]

In [6]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,device_id,event_key,devicemodel,language
0,2023-06-30 09:00:00,6a9bbd4a-7427-401c-80b3-6d82161ccb3b,$session_end,SM-A536N,ko
1,2023-06-30 09:00:00,dfea6e11-6022-4bfa-800b-61b87dff0ac1,view_timeline_tap,SM-N971N,ko
2,2023-06-30 09:00:00,ebb00523-9d18-445c-97ee-239dd8a68e59,click_question_open,"iPhone13,1",ko-KR
3,2023-06-30 09:00:00,9652674b-57ea-4d32-8f9c-21c11308aaef,view_home_tap,SM-M536S,ko
4,2023-06-30 09:00:00,40b75183-c3f4-4173-ab6f-9bc64cbc70de,view_questions_tap,"iPhone14,4",ko-KR


In [7]:
df_simple.to_parquet("/home/leesh/codeit_project/codeit-project-docker/parquet/230629.parquet", engine="pyarrow", compression="snappy")